# 🌿 Leaf Health Detection V2 (YOLOv8m + Advanced Analysis)

This is the **Version 2** code for RubberSense Leaf Detection.
**Features:**
1.  **YOLOv8m (Medium)** Classification for higher accuracy.
2.  **Leaf Isolation**: Analyzes *only* the leaf area (ignoring background).
3.  **Disease Type Detection**: Identifies specific diseases based on your dataset folders.
4.  **Color Detection**: meaningful dominant color analysis.
5.  **Spot Counting**: Advanced contour/blob detection for spots.

In [ ]:
# 1. Install Dependencies
!pip install ultralytics opencv-python-headless matplotlib numpy

In [ ]:
import os
import cv2
import numpy as np
import zipfile
import matplotlib.pyplot as plt
from ultralytics import YOLO
from sklearn.cluster import KMeans
from google.colab import drive
from google.colab.patches import cv2_imshow # For showing images in Colab

# Mount Drive
drive.mount('/content/drive')

## 2. Dataset Setup
This script automatically unzips your dataset from Google Drive.
**Structure required inside the ZIP:**
```
train/
  Healthy/
  Powdery_Mildew/
  ...
val/
  Healthy/
  Powdery_Mildew/
  ...
```

In [ ]:
# --- CONFIGURATION ---
import glob

# Try to find the zip file automatically (case insensitive search)
# This looks for any zip file containing 'Leaf' in MyDrive
possible_files = glob.glob('/content/drive/MyDrive/*Leaf*.zip')

if possible_files:
    ZIP_PATH = possible_files[0] # Pick the first one found
    print(f"Found zip file: {ZIP_PATH}")
else:
    # Fallback to specific filenames if glob fails
    ZIP_PATH = '/content/drive/MyDrive/Final_Leaf_Datasets.zip'
    print(f"Zip file not found via search. Trying specific path: {ZIP_PATH}")

EXTRACT_DIR = '/content/dataset_raw'  # Temporary extraction folder
FINAL_DATASET_DIR = '/content/dataset_ready' # Where we will organize files for YOLO

# 1. Unzip Logic
import shutil
import random
from pathlib import Path

if os.path.exists(ZIP_PATH):
    if not os.path.exists(EXTRACT_DIR):
        print(f"Unzipping {ZIP_PATH} to {EXTRACT_DIR}...")
        with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
            zip_ref.extractall(EXTRACT_DIR)
        print("Unzip complete!")
    else:
        print("Dataset already extracted.")
else:
    print(f"Error: Zip file not found at {ZIP_PATH}. Please check the path.")

# 2. Dataset Organizer (The Magic Step)
# This converts your nested folders (Diseased/Disease-Leaf_1/...) into YOLO format (train/val)

def organize_dataset(source_dir, target_dir, split_ratio=0.8):
    print("Organizing dataset...")
    
    # Define classes we want to find
    target_classes = ['Diseased', 'Healthy']
    
    # Create standardized folders
    for split in ['train', 'val']:
        for cls in target_classes:
            os.makedirs(os.path.join(target_dir, split, cls), exist_ok=True)
            
    # Walk through all files
    image_extensions = {'.jpg', '.jpeg', '.png', '.bmp'}
    file_counts = {cls: 0 for cls in target_classes}
    
    for root, dirs, files in os.walk(source_dir):
        for file in files:
            if os.path.splitext(file)[1].lower() in image_extensions:
                file_path = os.path.join(root, file)
                
                # Determine class based on folder path
                # We look for 'Diseased' or 'Healthy' in the path
                found_class = None
                for cls in target_classes:
                    # Check if class name is part of the path (case insensitive)
                    if cls.lower() in file_path.lower():
                        found_class = cls
                        break
                
                if found_class:
                    # Random split
                    split = 'train' if random.random() < split_ratio else 'val'
                    dest_path = os.path.join(target_dir, split, found_class, file)
                    
                    # Handle duplicate filenames by renaming
                    if os.path.exists(dest_path):
                        base, ext = os.path.splitext(file)
                        dest_path = os.path.join(target_dir, split, found_class, f"{base}_{random.randint(1000,9999)}{ext}")
                    
                    shutil.copy2(file_path, dest_path)
                    file_counts[found_class] += 1

    print(f"Organization Complete! Found images: {file_counts}")
    
    if sum(file_counts.values()) == 0:
        print("\n❌ ERROR: No images found to organize!")
        print(f"Checked directory: {source_dir}")
        print("Did the zip file extract correctly? Here is what is inside:")
        for root, dirs, files in os.walk(source_dir):
             print(f"{root} -> {len(files)} files")
        # Create dummy file to prevent immediate crash, but user needs to fix dataset
    
    return target_dir

# Run the organizer
DATASET_ROOT = organize_dataset(EXTRACT_DIR, FINAL_DATASET_DIR)

MODEL_SIZE = 'yolov8m-cls.pt'  # Using MEDIUM model as requested
EPOCHS = 25
IMG_SIZE = 224

## 3. Train YOLOv8m Model

In [ ]:
# Load YOLOv8 Medium Classification Model
model = YOLO(MODEL_SIZE)

# Train the model
# We point 'data' to the folder where we extracted the zip
results = model.train(
    data=DATASET_ROOT,
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    project='/content/drive/MyDrive/RubberSense/runs',
    name='leaf_disease_v2_medium'
)

## 4. Advanced Analysis Logic (Color, Spots, Isolation)

In [ ]:
class LeafAnalyzer:
    def __init__(self, model_path):
        self.model = YOLO(model_path)
    
    def get_dominant_color(self, img, mask):
        """Extracts dominant color ONLY from the masked leaf area."""
        # Apply mask to get only leaf pixels
        masked_img = cv2.bitwise_and(img, img, mask=mask)
        
        # Flatten pixels where mask is not 0
        pixels = masked_img[mask > 0]
        
        if len(pixels) == 0:
            return "Unknown"

        # Use K-Means to find main color
        kmeans = KMeans(n_clusters=1)
        kmeans.fit(pixels)
        dominant_color_bgr = kmeans.cluster_centers_[0]
        
        # Convert to readable name (Simplified)
        b, g, r = dominant_color_bgr
        if g > r and g > b:
            if r > 100 and g > 100: return "Yellow-Green"
            return "Green"
        elif r > g and r > b:
            return "Brown/Red"
        elif r > 150 and g > 150:
            return "Yellow"
        else:
            return "Dark/Grey"

    def isolate_leaf(self, img):
        """Generates a mask for the leaf (Green/Yellow/Brown hues)."""
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        
        # Green Mask
        lower_green = np.array([25, 40, 40])
        upper_green = np.array([90, 255, 255])
        mask_green = cv2.inRange(hsv, lower_green, upper_green)
        
        # Brown/Yellow Mask (for diseased parts)
        lower_brown = np.array([10, 50, 50])
        upper_brown = np.array([25, 255, 255])
        mask_brown = cv2.inRange(hsv, lower_brown, upper_brown)
        
        # Combine masks
        mask = cv2.bitwise_or(mask_green, mask_brown)
        
        # Clean up noise
        kernel = np.ones((5,5), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        
        return mask

    def count_spots(self, img, leaf_mask):
        """Counts dark spots WITHIN the leaf mask."""
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Invert: Dark spots become bright
        gray_inv = cv2.bitwise_not(gray)
        
        # Thresholding for spots
        _, thresh = cv2.threshold(gray_inv, 180, 255, cv2.THRESH_BINARY)
        
        # Ensure we ONLY count spots ON the leaf (apply leaf mask)
        thresh = cv2.bitwise_and(thresh, thresh, mask=leaf_mask)
        
        # Find contours
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Filter by size (avoid noise)
        valid_spots = [cnt for cnt in contours if 10 < cv2.contourArea(cnt) < 500]
        
        return len(valid_spots), valid_spots

    def analyze(self, image_path):
        img = cv2.imread(image_path)
        if img is None: return
        img = cv2.resize(img, (640, 640))
        
        # 1. AI Classification (Disease Type)
        results = self.model(img, verbose=False)
        probs = results[0].probs
        class_name = results[0].names[probs.top1]
        confidence = float(probs.top1conf) * 100
        
        # 2. Isolate Leaf
        leaf_mask = self.isolate_leaf(img)
        
        # 3. Detect Color (on leaf only)
        color_name = self.get_dominant_color(img, leaf_mask)
        
        # 4. Count Spots (on leaf only)
        spot_count = 0
        spot_contours = []
        if class_name.lower() != 'healthy':
            spot_count, spot_contours = self.count_spots(img, leaf_mask)
        
        # --- VISUALIZATION ---
        vis_img = img.copy()
        
        # Draw Mask Outline (Blue)
        mask_contours, _ = cv2.findContours(leaf_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(vis_img, mask_contours, -1, (255, 0, 0), 2)
        
        # Draw Spots (Red)
        cv2.drawContours(vis_img, spot_contours, -1, (0, 0, 255), -1)
        
        # Text Info
        text = f"{class_name.upper()} ({confidence:.1f}%)\nColor: {color_name} | Spots: {spot_count}"
        y0, dy = 30, 30
        for i, line in enumerate(text.split('\n')):
            y = y0 + i * dy
            cv2.putText(vis_img, line, (10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            
        print(f"Diagnosis: {class_name}")
        print(f"Leaf Color: {color_name}")
        print(f"Spots Detected: {spot_count}")
        cv2_imshow(vis_img)

In [ ]:
# 5. Run Inference on Test Image
# Point this to your trained model path after training
TRAINED_MODEL_PATH = '/content/drive/MyDrive/RubberSense/runs/leaf_disease_v2_medium/weights/best.pt'

# Initialize Analyzer
if os.path.exists(TRAINED_MODEL_PATH):
    analyzer = LeafAnalyzer(TRAINED_MODEL_PATH)
    
    # Test on an image - change this to point to a file inside the extracted dataset
    # Example: '/content/dataset/val/Powdery_Mildew/some_image.jpg'
    TEST_IMG = '/content/dataset/val/Powdery_Mildew/test_leaf.jpg' 
    if os.path.exists(TEST_IMG):
        analyzer.analyze(TEST_IMG)
    else:
        print("Please provide a valid test image path.")
else:
    print("Model training not finished or path incorrect.")